# Hacka (midterm) thon 

## Detecting Malicious URLs 

Today you are invited to repeat the path of researchers Detecting Malicious URLs.
An anonymized 120-day subset of our ICML-09 data set.
The data set consists of about 2.4 million URLs (examples) and 3.2 million features. 

#### 1. Download data using link below
[Download Dataset](http://www.sysnet.ucsd.edu/projects/url/url_svmlight.tar.gz)

#### 2. Description of Data (SVM-light)
Uncompressing the archive url_svmlight.tar.gz will yield a directory url_svmlight/ containing the following files:

1. **FeatureTypes**. A text file list of feature indices that correspond to real-valued features.
2. **DayX.svm** (where X is an integer from 0 to 120) --- The data for day X in SVM-light format. A label of +1 corresponds to a malicious URL and -1 corresponds to a benign URL.


#### 3. Read article
Please familiarize yourself with original research article. It will give you required context.

*"**Beyond Blacklists: Learning to Detect Malicious Web Sites from Suspicious URLs**"* 

*Justin Ma, Lawrence K. Saul, Stefan Savage, Geoffrey M. Voelker* 

## Demo part

#### 1. Upload data

In [1]:
import glob
import matplotlib.pyplot as plt
from sklearn.datasets import load_svmlight_file
files = glob.glob('./url_svmlight/*.svm')
print("There are %d files" % len(files))
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

There are 121 files


#### 2. What is inside

In [3]:
import tarfile
from sklearn.datasets import load_svmlight_file
import numpy as np

In [4]:
uri = ('./url_svmlight.tar.gz')
tar = tarfile.open(uri, "r:gz")
max_obs = 0
max_vars = 0
i = 0
split = 5
for tarinfo in tar:
    print("extracting %s,f size %s" % (tarinfo.name, tarinfo.size))
    if tarinfo.isfile():
        f = tar.extractfile(tarinfo.name)
        X,y = load_svmlight_file(f)
        max_vars = np.maximum(max_vars, X.shape[0])
        max_obs = np.maximum(max_obs, X.shape[1])
    if i > split:
        break
    i+=1
print("max X = %s, max y dimension = %s" % (max_obs, max_vars)) 

extracting url_svmlight,f size 0
extracting url_svmlight/Day33.svm,f size 18674876
extracting url_svmlight/Day32.svm,f size 18599211
extracting url_svmlight/Day53.svm,f size 18963938
extracting url_svmlight/Day20.svm,f size 18633460
extracting url_svmlight/Day7.svm,f size 18777054
extracting url_svmlight/Day117.svm,f size 18106370
max X = 3231952, max y dimension = 20000


#### 3. What is inside

In [5]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

classes = [-1,1] # 1_:url- safety, -1: url- non-safety
sgd = SGDClassifier(loss='log')
n_features = 3231952
split = 5
i = 0
for tarinfo in tar:
    if i > split:
        break
    if tarinfo.isfile():
        f = tar.extractfile(tarinfo.name)
        X,y = load_svmlight_file(f,n_features=n_features)
        if i < split:
            sgd.partial_fit(X,y, classes = classes)
        if i == split:
            print (classification_report(sgd.predict(X),y))
    i+=1

              precision    recall  f1-score   support

          -1       0.99      0.97      0.98     14567
           1       0.92      0.97      0.95      5433

    accuracy                           0.97     20000
   macro avg       0.96      0.97      0.96     20000
weighted avg       0.97      0.97      0.97     20000



## Midterm (Part 2)

### Grading criteria
- Complete solution - 60%
- F1 Score - 40%
    - The first 10 results get 40%
    - Worst result get 20%
    - All others are on a scale between them

### Deadline
20:00 MSK, April 4

#### 1. Train, test
- Upload data (you can use template above)
- Separate your dataset into train and test subsets of observations
- Use the 8:2 ratio: 80% train set, 20% test set

In [2]:
from random import random
def train_test_split(train_size = 0.8):
    
    train = open('./url_svmlight/train.svm','w')
    test  = open('./url_svmlight/test.svm','w')
    summ = 0
    for i in range(121):
        inn = open('./url_svmlight/Day' + str(i) + '.svm','r')
        
        print("file: " + str(i))
        
        q = [0, 0]
        
        for line in inn:
            a = line.split()
            if(a[0] == "-1"):
                q[0] += 1
            else:
                q[1] += 1
        
        inn.close()
        summ += q[0] + q[1]
        check = [int(train_size * q[0]),int(train_size * q[1])]
        start = [0, 0]
        
        inn = open('./url_svmlight/Day' + str(i) + '.svm','r')
        
        if (train_size * q[0]) % 1 >=0.5:
            check[0] += 1
        if (train_size * q[1]) % 1 >=0.5:
            check[1] += 1
        for line in inn:
            a = line.split()

            rand = random()
            if rand > 0.5:
                if a[0] == "-1":
                    if start[0] + 1 <= check[0]:
                        train.write(line)
                        train.write("\n")
                        start[0] += 1
                    else:
                        test.write(line)
                        test.write("\n")
                        q[0] -= 1
                else:
                    if start[1] + 1 <= check[1]:
                        train.write(line)
                        train.write("\n")
                        start[1] += 1
                    else:
                        test.write(line)
                        test.write("\n")
                        q[1] -= 1
            else:
                if(a[0] == "-1"):
                    if q[0] > check[0]:
                        test.write(line)
                        test.write("\n")
                        q[0] -= 1
                    else :
                        train.write(line)
                        train.write("\n")
                        start[0] += 1
                else:
                    if q[1] > check[1]:
                        test.write(line)
                        test.write("\n")
                        q[1] -= 1
                    else :
                        train.write(line)
                        train.write("\n")
                        start[1] += 1
        print("finish: " + str(i))
    print(summ)

In [7]:
from sklearn.datasets import load_svmlight_file
from sklearn.linear_model import Perceptron
import numpy as np

train_test_split()

data = None
n_features = 3231961


data, target = load_svmlight_file("./url_svmlight/train.svm",n_features=n_features)

file: 0
finish: 0
file: 1
finish: 1
file: 2
finish: 2
file: 3
finish: 3
file: 4
finish: 4
file: 5
finish: 5
file: 6
finish: 6
file: 7
finish: 7
file: 8
finish: 8
file: 9
finish: 9
file: 10
finish: 10
file: 11
finish: 11
file: 12
finish: 12
file: 13
finish: 13
file: 14
finish: 14
file: 15
finish: 15
file: 16
finish: 16
file: 17
finish: 17
file: 18
finish: 18
file: 19
finish: 19
file: 20
finish: 20
file: 21
finish: 21
file: 22
finish: 22
file: 23
finish: 23
file: 24
finish: 24
file: 25
finish: 25
file: 26
finish: 26
file: 27
finish: 27
file: 28
finish: 28
file: 29
finish: 29
file: 30
finish: 30
file: 31
finish: 31
file: 32
finish: 32
file: 33
finish: 33
file: 34
finish: 34
file: 35
finish: 35
file: 36
finish: 36
file: 37
finish: 37
file: 38
finish: 38
file: 39
finish: 39
file: 40
finish: 40
file: 41
finish: 41
file: 42
finish: 42
file: 43
finish: 43
file: 44
finish: 44
file: 45
finish: 45
file: 46
finish: 46
file: 47
finish: 47
file: 48
finish: 48
file: 49
finish: 49
file: 50
finish: 50


In [24]:
n_features = 3231961
data, target = load_svmlight_file("./url_svmlight/train.svm",n_features=n_features)

In [25]:
print(data.shape)

(1916904, 3231961)


#### 2. Find out whether it is possible to reduce the dimension?

In [26]:
# YOUR CODE HERE
# Yes, we could. However, the initial paper proposed online learning instead of batch. Moreover, it does not contain mentions of dimensionality reduction.
# However, since we are using batch learning we might want to do that to reduce dimensionality. For instance, soft confidence weighted algorithm cannot perform 
# computations on such a large dataset.
# Comments: We reduced the initial dataset to 100 features using Truncated SVD. It captures 60.5% of explained variance ratio.
# Truncated SVD should perform better on large sparse datasets which cannot be centered without making the memory usage explode.
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=0)
svd.fit(data)

print(svd.explained_variance_ratio_)

print(svd.explained_variance_ratio_.sum())

print(svd.singular_values_)

[0.01891587 0.10121791 0.05826583 0.02598372 0.02237064 0.02065406
 0.02027658 0.01812224 0.01594569 0.01300335 0.01200047 0.01116721
 0.01082642 0.01018591 0.0098815  0.00910171 0.00869914 0.00782822
 0.0074706  0.00727178 0.00677976 0.00632283 0.00625342 0.00601369
 0.00575902 0.00536761 0.00510548 0.00500917 0.00479991 0.00429118
 0.00411876 0.00402973 0.00378781 0.00356946 0.00340468 0.00325698
 0.00319533 0.00316682 0.00315137 0.0029938  0.00287629 0.00284309
 0.002793   0.00273524 0.00273223 0.00265768 0.00264986 0.00259308
 0.00254209 0.00246111 0.00243054 0.00236821 0.00231311 0.00227587
 0.00222463 0.00215692 0.00212117 0.00208857 0.00206884 0.00202759
 0.0019657  0.00190219 0.00189828 0.00184138 0.00183582 0.00176562
 0.00172946 0.00169214 0.00166553 0.00160721 0.00159657 0.0015808
 0.00152448 0.00151846 0.00150706 0.00148017 0.00146664 0.00142565
 0.0014191  0.00141386 0.00139357 0.00138097 0.00134218 0.00132144
 0.00128823 0.00127778 0.00125667 0.00124371 0.00122603 0.00119

In [27]:
svd_transform = svd.transform(data)

In [28]:
print(svd_transform.shape)

(1916904, 100)


#### 3. Create a model

In [ ]:
!pip install scw

In [29]:
from scw import SCW1, SCW2
import numpy as np
# f = np.array(X)
# print(f)
# print(np.ndim(f))
scw = SCW1(C=1.0, ETA=1.0)
scw.fit(svd_transform, target)
y_pred = scw.predict(svd_transform)

#### 4. Get the quality
- precision
- recall
- f1-score
- support 

In [30]:
from sklearn.metrics import classification_report

data, target = load_svmlight_file("./url_svmlight/test.svm",n_features=n_features)
test_svd_transform = svd.transform(data)
print (classification_report(scw.predict(test_svd_transform),target, digits = 6))
# test_svd_transform = svd.transform(data)

              precision    recall  f1-score   support

          -1   0.978132  0.973317  0.975719    322382
           1   0.945705  0.955274  0.950465    156844

    accuracy                       0.967412    479226
   macro avg   0.961919  0.964296  0.963092    479226
weighted avg   0.967519  0.967412  0.967454    479226

